## Inner Functions & Decorators

### Simple Inner Function

Inner function is a function defined inside another function, for example:

In [ ]:
def outer():
    def inner():
        print('from inner')
    inner()
    print('from outer')
outer()
# from inner
# from outer

The inner function can use the variables of the outer function.

In [ ]:
def outer(a):
    local_var = 'world'
    def inner():
        print(a, local_var)
    inner()

outer('hello')
# hello world

The inner function can not be called outside the outer function. Therefore the implementation is hidden from outside, like a private function.

If you just want to hide the implementation, please use private function instead of inner function.

In [ ]:
def outer(a):
    local_var = 'world'
    def inner():
        print(a, local_var)
    inner()

inner()
# NameError: name 'inner' is not defined

### Closure

There is a important feature about inner function: __closure__.

A closure is an inner function returned by the outer one and the inner function uses some local variables of the outer one. For example:

In [ ]:
def outer(a):
    local_var = 'from closure'
    def inner(b):
        print(a, b, local_var)
    return inner

func = outer('hello')
func('world')
# hello world from closure
func('groot')
# hello groot from closure

The local variables of outer function used in inner function keeps their values. We can inspect those variables.

In [ ]:
import inspect
inspect.getclosurevars(func)
# ClosureVars(nonlocals={'a': 'hello', 'local_var': 'from closure'}, globals={}, builtins={'print': <built-in function print>}, unbound=set())

An interesting case is, what will happen if there is an undeclared variable in the inner function ?

In [ ]:
def outer(a):
    local_var = 'from closure'
    def inner(b):
        print(a, b, c, local_var)
    return inner

In [ ]:
func = outer('I')
func('am')
# NameError: name 'c' is not defined

inspect.getclosurevars(func)
# ClosureVars(nonlocals={'a': 'I', 'local_var': 'from closure'}, globals={}, builtins={'print': <built-in function print>}, unbound={'c'})

Of course, the `c` is not defined. Now let's declare the `c` in the global scope, and call the `func` again.

In [ ]:
c = 'groot'
func('am')
# I am groot from closure

inspect.getclosurevars(func)
# ClosureVars(nonlocals={'a': 'I', 'local_var': 'from closure'}, globals={'c': 'groot'}, builtins={'print': <built-in function print>}, unbound=set())

Python tries to resolve unbound variables from the current global scope.

One usage of closure is factory function:

In [ ]:
def generate_power(exponent):
    def power(base):
        return base ** exponent
    return power

raise_two = generate_power(2)
raise_two(4)
# 16
raise_three = generate_power(3)
raise_three(4)
# 64

Another important usage is "decorator".

### Decorator (with function)

Similar to ["decorator" pattern](https://en.wikipedia.org/wiki/Decorator_pattern) which intends to add additional behaviors before and after the current behaviors.

In python, "decorator" is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

For example, we want to add some log before and after calling a function.

In [ ]:
def log_func(func):
    def wrapper(*args, **kwargs):
        print('something before')
        func(*args, **kwargs)
        print('somethine after')
    return wrapper

def hello():
    print(f'hi')

hello = log_func(hello)
hello()
# something before
# hi
# somethine after

Additional, python gives a syntax sugar: 

`@decorator` <==> `func = decorator(func)`

In [ ]:
hello = log_func(hello)

# equivalent to
@log_func
def hello():
    print(f'hi')

hello()
# something before
# hi
# somethine after

But there is a problem with the above function, we lost the original function's identity.

In [ ]:
hello
# <function __main__.log_func.<locals>.wrapper(*args, **kwargs)>
hello.__name__
# wrapper

The function's name is no longer `hello` but is `wrapper` which is the inner function's name.

To handle this, we need to use `functools.wraps`.

In [ ]:
import functools

def log_func(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('something before')
        func(*args, **kwargs)
        print('somethine after')
    return wrapper

@log_func
def hello():
    print(f'hi')

hello
# <function __main__.hello()>
hello.__name__
# hello

Sometimes we need to give some arguments to the decorator function. For example: 

In [ ]:
@log_func(before_only=True)
def hello():
    print(f'hi')

# equivalent to
# hello = log_func(before_only=True)(hello)

To achieve this goal, we need to add one more level of function.

In [ ]:
import functools
def log_func(before_only=False):
    def _log_func(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('something before')
            func(*args, **kwargs)
            if not before_only:
                print('somethine after')
        return wrapper
    return _log_func


In [ ]:
@log_func(before_only=True)
def hello():
    print(f'hi')

hello()
# something before
# hi

But it breaks the origin version of decorator:

In [ ]:
@log_func
def hello():
    print(f'hi')

hello()
# TypeError: _log_func() missing 1 required positional argument: 'func'

We have to use decorator differently.

In [ ]:
@log_func()  # note this extra '()'
def hello():
    print(f'hi')

hello()
# something before
# hi
# somethine after

It is a little inconvient. I suggest to use kwargs only arguments to overcome this problem.

In [ ]:
import functools
# the parameter after `*` is kwargs only, cf. https://peps.python.org/pep-3102/
def log_func(func=None, *, before_only=False):
    def _log_func(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('something before')
            func(*args, **kwargs)
            if not before_only:
                print('somethine after')
        return wrapper
    if func is not None:
        return _log_func(func)
    return _log_func

In [ ]:
@log_func
def hello():
    print(f'hi')

hello()
# something before
# hi
# somethine after

In [ ]:
@log_func(before_only=True)
def hello():
    print(f'hi')

hello()
# something before
# hi

### Decorator (with class)

To recall, the decorator syntax `@deco` is equivalent to `func = deco(func)`. The `deco` object does not have to be a function, any thing can be used as `deco`, if:
- `deco` is a callable which accepts a variable
- `deco` returns a callable

For instance, if a class defines a dunder method `__call__`, it could be used as a decorator.

The `__call__` method can make instance of the class to be a callable.

In [ ]:
class CallMe:
    def __call__(self):
        print('hello')

iamcallable = CallMe()
iamcallable()
# hello

Therefore, the class based decorator could look like this (this is one possible variation):

In [ ]:
import functools
class LogCall:
    def __init__(self, func=None, *, before_only=False):
        self._before_only = before_only
        if func is not None:
            self._wrapper_func = self._log_func(func)
        else:
            self._wrapper_func = self._log_func

    def __call__(self, *args, **kwargs):
        return self._wrapper_func(*args, **kwargs)

    def _log_func(self, func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print('something before')
            func(*args, **kwargs)
            if not self._before_only:
                print('somethine after')
        return wrapper

@LogCall
def hello():
    print(f'hi')

hello()
# something before from class
# hi
# something after from class

@LogCall(before_only=True)
def hello():
    print(f'hi')

hello()
# something before from class
# hi
